# Ignore files

This page specifies aspects of how exactly `.dockerignore` works.

## Access to ignore files

If you try to interact somehow with ingnored files during build - you'll get error from docker.

---

The following cell creates:

- `dockerfile` that tries to copy `folder` and pring it's content during container start.
- `.dockerignore` that counts `folder`.
- And `folder` that contains `file`.

In [4]:
mkdir folder
echo "message" > folder/file

# .dockerignore
cat << EOF > .dockerignore
folder
EOF

# dockerfile
cat << EOF > dockerfile
FROM alpine
COPY folder folder
CMD ["ls", "folder"]
EOF

If you try to build an image from such a configuration, you'll get an error - `"/folder": not found` in the `COPY` step. This happens because `.dockerignore` has banned this file.

In [5]:
docker build -t test_image .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (3/6)                                          docker:default
 => [internal] load build definition from dockerfile                       0.1s
 => => transferring dockerfile: 89B                                        0.0s
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 47B                                           0.0s
 => [internal] load metadata for docker.io/library/alpine:latest           0.0s
[+] Building 0.2s (6/6) FINISHED                                 docker:default
 => [internal] load build definition from dockerfile                       0.1s
 => => transferring dockerfile: 89B                                        0.0s
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 47B                                           0.0s
 => [internal] load metadata for docker.

: 1

Ok, let's fix `.dockerignore` to reject `folder/ignored_file`. Add such a file, and another `folder/new_file`.

In [6]:
rm -r .dockerignore dockerfile folder

## Ignore only for build

Note that `.dockerignore` only affects the build phase. You can still copy an ignored file or mount it as a volume.

---

In the following example, let's say we want to hide `secret_file` from the Docker containers. Adding it to the `.dockerignore` file will help achieve this.

In [9]:
echo "secret info" >  secret_file

cat << EOF > .dockerignore
secret_file
EOF

cat << EOF > dockerfile
FROM alpine
EOF

docker build -t temp_image . &> /dev/null

Now, let's try mounting `secret_file` as `mounted_file` and copying it to the container as `copied_file`.

In [10]:
docker run -itd --rm --name example -v ./secret_file:/mounted_file temp_image
docker cp secret_file example:copied_file

c8d89266e0ade3b0dc0f75c62fdbb64e6d585dc9efd7fe223d907832bf2edbb1
Successfully copied 2.05kB to example:copied_file


Everything works fine, and we can easily access the information from the container.

In [11]:
docker exec example cat copied_file
docker exec example cat mounted_file

secret info
secret info


Keep environment clean.

In [12]:
docker stop example
docker rmi temp_image
rm dockerfile .dockerignore secret_file

example
Untagged: temp_image:latest


## Subfolders ignore

Using the .gitignore analogy, one might think that if .dockerignore is in a subdirectory of an assembly, it would apply to files in that directory. In this section we will check if this is the case.

---

The following example attempts to hide the `temp_folder/secret_file` file from the `docker build`. We're using a docker image that allows us to check what files are in `temp_folder`.

In [24]:
mkdir temp_folder
echo "secret message" > temp_folder/secret_file

cat << EOF > temp_folder/.dockerignore
secret_file
EOF

cat << EOF > dockerfile
FROM alpine
COPY . /context/
CMD ["ls", "-a" , "context/temp_folder"]
EOF

docker build -t temp_image . &> /dev/null
docker run --rm temp_image

.
..
.dockerignore
secret_file


As a result, `secret_file` is in the context of the docker container. So it **doesn't work**.

Keeping the environment clean.

In [23]:
docker rmi temp_image
rm -r dockerfile temp_folder

Untagged: temp_image:latest
Deleted: sha256:20151632e46738b5b016b418b1be56a125c6983a43d761ad49691ef6defd3896
